**Import necessary packages/modules**

In [ ]:
# Cell 1
from pathlib import Path

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from google.colab import drive
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PolynomialFeatures

**Connect this notebook to your Google Drive**

In [ ]:
# Cell 2
drive.mount("/content/gdrive", force_remount=True)
notebook_path = Path("/content/gdrive/MyDrive/wdts-psi")
notebook_path /= Path("Session 06 - Computational Physics")
notebook_path

**Generate a `numpy array` from a CSV (comma separated value) formatted text file**
1. The 1st column (index 0) is the time (in seconds)
2. The 2nd column (index 1) is the distance (in meters)

In [ ]:
# Cell 3
file_name = "kinematics_regression.csv"
file_path = notebook_path / file_name
data = np.genfromtxt(file_path, skip_header=1, delimiter=",")
pd.DataFrame(data, columns=["Time (s)", "Distance (m)"])

**Create a function to fit a quadratic polynomial to the data**
1. The fitted curve has the form $y=Ax^2+Bx+C$
2. Use the `LinearRegression()` class from the <b>scikit-learn</b> package
3. The independent variable is in vector $x$ (a 1-D array)
4. The dependent variable is in vector $y$ (a 1-D array)

In [ ]:
# Cell 4
def fit_quadratic(x, y):
    # Reshape vector x to become matrix x
    x = x[:, np.newaxis]
    transformer = PolynomialFeatures(degree=2, include_bias=False)
    transformer.fit(x)
    # The matrix x2 will have two columns:
    # 1) the original x values and 2) the x**2 values
    x2 = np.array(transformer.transform(x))
    model = LinearRegression().fit(x2, y)
    a = model.coef_[1]
    b = model.coef_[0]
    c = model.intercept_
    return a, b, c

**Fit the data in the file to the quadratic curve**
1. From Newtonian kinematics: $s=\frac{1}{2}{at}^2+v_0 t + s_0$
2. Mapping this to $y=Ax^2+Bx+C$ (where $x=t$) yields $A=\frac{1}{2}a$
3. Therefore, the acceleration is $a=2A$ and the initial velocity is $v_0=B$

In [ ]:
# Cell 5
vec_x, vec_y = data.T
a, b, c = fit_quadratic(vec_x, vec_y)

print(f"Constant acceleration = {a * 2:.4f} m/s^2")
print(f"Initial velocity = {b:.4f} m/s")

**Plot the trajectory data and the fitted quadratic curve**

In [ ]:
# Cell 6
x = np.linspace(np.min(vec_x), np.max(vec_x))
plt.plot(x, a * x**2 + b * x + c)
plt.scatter(vec_x, vec_y, color="red")
plt.title("Newtonian Kinematics")
plt.xlabel("Time (sec)")
plt.ylabel("Distance (m)")
plt.show()